## Imports

In [2]:
import torch
import pandas as pd
import random
#import wandb
import copy
from tqdm import tqdm
import numpy as np

In [6]:
torch.backends.mps.is_built()

True

## GPU detection

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device} {[torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())]}")

device: cpu []


## Variables

In [3]:
best_weights_path = 'data/best_weights.pt'
csv_path = 'data/encoder_training.csv'
batch_size = 64
learning_rate = 0.001
es_patience = 5
es_min_delta = 0.01
num_epochs = 100

## Dataloader

### Dataset

In [4]:
class EncoderTrainingDataset(torch.utils.data.Dataset):
    def __init__(self, csv_path):
        self.csv_path = csv_path
        #read csv with columns precipitation, evaporation, grace, level
        self.df = pd.read_csv(csv_path, names=['precipitation, evaporation, grace'])
    def __getitem__(self, index):
        #read csv with columns precipitation, evaporation, grace, level
        row = self.df.iloc[index]
        #return precipitation, evaporation, grace
        return torch.tensor(row['precipitation, evaporation, grace'], dtype=torch.float)
    def __len__(self):
        return len(self.df)

### Dataloader

In [5]:
dataset = EncoderTrainingDataset(csv_path)
random.seed(0)
train_indices = random.sample(range(len(dataset)), int(len(dataset) * 0.8))
train_dataset = torch.utils.data.Subset(dataset,train_indices)
valid_indicies = list(set(range(len(dataset))) - set(train_indices))
valid_dataset = torch.utils.data.Subset(dataset, valid_indicies)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

FileNotFoundError: [Errno 2] No such file or directory: 'data/encoder_training.csv'

## Model

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(3, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU()
        )
    def forward(self, x):
        return self.model(x)
    
class Decoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 3),
            torch.nn.ReLU()
        )
    def forward(self, x):
        return self.model(x)

#combine encoder and decoder into autoencoder
class Autoencoder(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = Encoder()
        self.decoder = Decoder()
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
model = Autoencoder().to(device) #move model to gpu
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
def train():
  try:
    if es_patience >= 0:
      best_model_wts = copy.deepcopy(model.state_dict())
      best_loss = float('inf')
      es_no_improvement = 0
    #iterate over epochs
    for epoch in range(num_epochs):
      print(f'\n\033[1m\033[94mEpoch {epoch}/{num_epochs}\033[0m\033[0m')
      
      # training
      print('\033[92mTraining...\033[0m')
      model.train() # enable gradient calculation
      epoch_train_loss = []
      for x in tqdm(train_dataloader): #iterate over batches
        x = x.to(device) #send data to gpu
        optimizer.zero_grad() #reseting gradients before each batch
        y = model(x) #forwardpropagation
        loss = torch.nn.functional.mse_loss(y, x).to(device)
        loss.backward() #backpropagation to calculate gradients
        epoch_train_loss.append(loss.item())
        optimizer.step() #apply gradients to update weights

      epoch_train_loss = np.mean(epoch_train_loss)
      wandb.log({f"train/loss": epoch_train_loss})
      print(f"train/loss: {epoch_train_loss}")
      
      # validating
      print('\033[92mValidating...\033[0m')
      model.eval()
      epoch_valid_loss = []
      #iterate over batches
      for x in tqdm(valid_dataloader):
        x = x.to(device)
        y = model(x)
        loss = torch.nn.functional.mse_loss(y, x).to(device)
        epoch_valid_loss.append(loss.item())
        
      epoch_valid_loss = np.mean(epoch_valid_loss)
      wandb.log({f"valid/loss": epoch_valid_loss})
      print(f"valid/loss: {epoch_valid_loss}")
      
      if es_patience >= 0:
        es_delta = best_loss - epoch_valid_loss
        if es_delta > es_min_delta:
          es_no_improvement = 0
          print(f"Validation loss improved by {es_delta}. Saving best model.")
          best_loss = epoch_valid_loss
          best_model_wts = copy.deepcopy(model.state_dict())
          torch.save(best_model_wts,best_weights_path)
        else:
          es_no_improvement += 1
          print(f"No loss improvement since {es_no_improvement}/{es_patience} epochs.")
        if es_no_improvement > es_patience:
          break

  except KeyboardInterrupt:
    print("Keyboard Interrupt")
  except BaseException as err:
    raise
  finally:
    if es_patience >= 0:
      print('Best rmse_valid: {:4f}'.format(best_loss))
      model.load_state_dict(best_model_wts)
  return model

In [ ]:
# Example wandb configuration. All experiments results are available at https://wandb.ai/rszostak/grace-pl
wandb.init(project="grace-pl", entity="rszostak", anonymous="must")
wandb.config = {
  "learning_rate": learning_rate,
  "batch_size": batch_size,
  "es_patience": es_patience,
  "es_min_delta": es_min_delta,
  "num_epochs": num_epochs,
  }

try:
  train()
finally:
  wandb.finish()